In [ ]:
from midiutil import MIDIFile
from mingus.core import chords

import os
import csv
import subprocess
import random

In [ ]:
NOTES = ["C", "C#", "D", "Eb", "E", "F", "F#", "G", "Ab", "A", "Bb", "B"]
OCTAVES = list(range(11))
NOTES_IN_OCTAVE = len(NOTES)

errors = {"notes": "Bad input, please refer this spec-\n"}

In [ ]:
def swap_accidentals(note):
    if note == "Db":
        return "C#"
    if note == "D#":
        return "Eb"
    if note == "E#":
        return "F"
    if note == "Gb":
        return "F#"
    if note == "G#":
        return "Ab"
    if note == "A#":
        return "Bb"
    if note == "B#":
        return "C"


In [ ]:
def note_to_number(note: str, octave: int) -> int:
    note = swap_accidentals(note)
    assert note in NOTES, errors["notes"]
    assert octave in OCTAVES, errors["notes"]

    note = NOTES.index(note)
    note += NOTES_IN_OCTAVE * octave

    assert 0 <= note <= 127, errors["notes"]

    return note


# fluidsynth_executable, '-i', C:\Work2_2566\Project2\Top 18 Free Piano Soundfonts\KAWAI good piano.sf2,      '-F', C:\Work2_2566\Project2\data_real\norest_1\output.wav, C:\Work2_2566\Project2\data_real\norest_1\midi_000.mid
def fs(input_path, output_path):
    fluidsynth_executable = "C:\\Users\\user\\fluidsynth-install\\bin\\fluidsynth.exe"
    soundfont_path = (
        "C:\\Work1_2567\\Project\\Top_18_Free_Piano_Soundfonts\\Porter_Grand_Piano.sf2"
    )
    midi_file = (
        input_path
    )
    output_file = (
        output_path
    )

    # Construct the command
    command = [
        fluidsynth_executable,
        "-i",
        soundfont_path,
        "-F",
        output_file,
        midi_file,
    ]

    try:
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")

    # delete midi file : keep only audio file
    os.remove(midi_file)

In [ ]:
array_of_notes = NOTES  # ตั้งว่าจะเอาโน๊ตอะไรบ้าง

array_of_note_numbers = []
array_temp = []
for note in range(len(array_of_notes)):
    array_temp = []
    OCTAVE = 4  # เปลี่ยน octive ตรงนี้
    array_of_note_numbers.append(note)
print(array_of_note_numbers)

track = 0
channel = 0
time = 0  # In beats
duration = 2  # In beats ตัวขาว
tempo = 120  # In BPM
volume = 255  # 0-127, as per the MIDI standard

# 4.2.1 จำนวน และลำดับของ pitch แตกต่างกัน

In [ ]:
def create_files_4_2_1(
    array_of_note_numbers,
    num_files,
    min_notes,
    max_notes,
    path,
    writer,
    rhythm,
    rest,
    track=0,
    channel=0,
    tempo=120,
    time=0, 
    duration=2,
    duration_rest=2,
    volume=255,
):
    generated_files = set()

    for _ in range(num_files):
        num_notes = random.randint(min_notes, max_notes)
        note_indices = random.sample(range(len(array_of_note_numbers)), num_notes)
        # note_indices.sort()  # Ensure notes are in ascending order
        notes_prefix = "".join([NOTES[i] for i in note_indices])
        note_indices_str = "".join(map(str, note_indices))

        if note_indices_str in generated_files:
            continue

        generated_files.add(note_indices_str)
        MyMIDI = MIDIFile(1)
        MyMIDI.addTempo(track, 0, tempo)
        rest = {
            # 4: "whole rest",
            2: "half rest",
            # 1: "Quarter rest",
            # 0.5: "eighth rest",
            # 0.25: "Sixteenth rest",
            0: "no rest",
        }
        all_rest = []
        time = 0
        for idx in note_indices:
            # duration_rest = random.choice([4, 2, 1, 0.5, 0.25])
            duration_rest = random.choice([2, 0])
            MyMIDI.addNote(
                track, channel, array_of_note_numbers[idx] + 48, time, duration, volume
            )
            time += duration
            time += duration_rest
            all_rest.append(rest[duration_rest])



        midi_file_name = f"{path}/midi_{notes_prefix+note_indices_str}.mid"
        with open(midi_file_name, "wb") as output_file:
            MyMIDI.writeFile(output_file)
        
        fs(f'{path}/midi_{notes_prefix+note_indices_str}.mid', f'{path}/wav_{notes_prefix+note_indices_str}.wav')
        label_indices = ",".join(map(str, note_indices))
        writer.writerow([f"wav_{notes_prefix+note_indices_str}.wav", f"[{label_indices}]", tempo, rhythm, num_notes, all_rest])


In [ ]:
# โน้ต 3 ตัว
rhythm = "all white"
rest = "all half rest"
# ระบุเส้นทางไปยังโฟลเดอร์ที่มีไฟล์ที่คุณต้องการเก็บข้อมูล
folder_path = "../../data/exp4-2/4-2-1"
# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
csv_file = "../../data/exp4-2/4-2-1/file_names_exp4-2-1.csv"

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "num of pitch"])

    path = "../../data/exp4-2/4-2-1"
    max_depth = 3  # we have three levels of loops in the original code
    create_files_4_2_1(array_of_note_numbers=array_of_note_numbers, num_files=1800, min_notes=3, max_notes=9, path=path, writer=writer, rhythm=rhythm, rest=rest)

print(f"File names have been written to {csv_file}")

# 4.2.2 จำนวน, ลำดับของ pitch และ tempo แตกต่างกัน

In [ ]:
def create_files_4_2_2(
    array_of_note_numbers,
    num_files,
    min_notes,
    max_notes,
    path,
    writer,
    rhythm,
    rest,
    track=0,
    channel=0,
    tempo=120,
    time=0, 
    duration=2,
    duration_rest=2,
    volume=255,
):
    generated_files = set()

    for _ in range(num_files):
        num_notes = random.randint(min_notes, max_notes)
        note_indices = random.sample(range(len(array_of_note_numbers)), num_notes)
        # note_indices.sort()  # Ensure notes are in ascending order
        notes_prefix = "".join([NOTES[i] for i in note_indices])
        note_indices_str = "".join(map(str, note_indices))
        tempo = random.choice([60, 80, 100, 120, 140, 160, 180])

        if note_indices_str in generated_files:
            continue

        generated_files.add(note_indices_str)
        MyMIDI = MIDIFile(1)
        MyMIDI.addTempo(track, 0, tempo)
        rest = {
            # 4: "whole rest",
            2: "half rest",
            # 1: "Quarter rest",
            # 0.5: "eighth rest",
            # 0.25: "Sixteenth rest",
            0: "no rest",
        }
        all_rest = []
        time = 0
        for idx in note_indices:
            # duration_rest = random.choice([4, 2, 1, 0.5, 0.25])
            duration_rest = random.choice([2, 0])
            MyMIDI.addNote(
                track, channel, array_of_note_numbers[idx] + 48, time, duration, volume
            )
            time += duration
            time += duration_rest
            all_rest.append(rest[duration_rest])



        midi_file_name = f"{path}/midi_{notes_prefix+note_indices_str}.mid"
        with open(midi_file_name, "wb") as output_file:
            MyMIDI.writeFile(output_file)
        
        fs(f'{path}/midi_{notes_prefix+note_indices_str}.mid', f'{path}/wav_{notes_prefix+note_indices_str}.wav')
        label_indices = ",".join(map(str, note_indices))
        writer.writerow([f"wav_{notes_prefix+note_indices_str}.wav", f"[{label_indices}]", tempo, rhythm, num_notes, all_rest])


In [ ]:
# โน้ต 3 ตัว
rhythm = "all white"
rest = "all half rest"
# ระบุเส้นทางไปยังโฟลเดอร์ที่มีไฟล์ที่คุณต้องการเก็บข้อมูล
folder_path = "../../data/exp4-2/4-2-2"
# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
csv_file = "../../data/exp4-2/4-2-2/file_names_exp4-2-2.csv"

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "num of pitch"])

    path = "../../data/exp4-2/4-2-2"
    max_depth = 3  # we have three levels of loops in the original code
    create_files_4_2_2(array_of_note_numbers=array_of_note_numbers, num_files=1800, min_notes=3, max_notes=9, path=path, writer=writer, rhythm=rhythm, rest=rest)

print(f"File names have been written to {csv_file}")

# 4.2.3 จำนวน, ลำดับของ pitch และ rhythm แตกต่างกัน

In [ ]:
def create_files_4_2_3(
    array_of_note_numbers,
    num_files,
    min_notes,
    max_notes,
    path,
    writer,
    rhythm,
    rest,
    track=0,
    channel=0,
    tempo=120,
    time=0, 
    duration=2,
    duration_rest=2,
    volume=255,
):
    generated_files = set()
    rhythm = {4:"whole", 2:"white", 1:"black", 0.5:"half", 0.25:"Quater"}
    for _ in range(num_files):
        num_notes = random.randint(min_notes, max_notes)
        note_indices = random.sample(range(len(array_of_note_numbers)), num_notes)
        # note_indices.sort()  # Ensure notes are in ascending order
        notes_prefix = "".join([NOTES[i] for i in note_indices])
        note_indices_str = "".join(map(str, note_indices))
        # tempo = random.choice([60, 80, 100, 120, 140, 160, 180])

        if note_indices_str in generated_files:
            continue

        all_rhythm = []
        generated_files.add(note_indices_str)
        MyMIDI = MIDIFile(1)
        MyMIDI.addTempo(track, 0, tempo)
        rest = {
            # 4: "whole rest",
            2: "half rest",
            # 1: "Quarter rest",
            # 0.5: "eighth rest",
            # 0.25: "Sixteenth rest",
            0: "no rest",
        }
        all_rest = []
        time = 0
        for idx in note_indices:
            duration = random.choice([4, 2, 1, 0.5, 0.25])
            # duration_rest = random.choice([4, 2, 1, 0.5, 0.25])
            duration_rest = random.choice([2, 0])
            MyMIDI.addNote(
                track, channel, array_of_note_numbers[idx] + 48, time, duration, volume
            )
            time += duration
            all_rhythm.append(rhythm[duration])

            time += duration_rest
            all_rest.append(rest[duration_rest])



        midi_file_name = f"{path}/midi_{notes_prefix+note_indices_str}.mid"
        with open(midi_file_name, "wb") as output_file:
            MyMIDI.writeFile(output_file)
        
        fs(f'{path}/midi_{notes_prefix+note_indices_str}.mid', f'{path}/wav_{notes_prefix+note_indices_str}.wav')
        label_indices = ",".join(map(str, note_indices))
        writer.writerow([f"wav_{notes_prefix+note_indices_str}.wav", f"[{label_indices}]", tempo, all_rest, num_notes, all_rest])


In [ ]:
# โน้ต 3 ตัว
rhythm = "all white"
rest = "all half rest"
# ระบุเส้นทางไปยังโฟลเดอร์ที่มีไฟล์ที่คุณต้องการเก็บข้อมูล
folder_path = "../../data/exp4-2/4-2-3"
# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
csv_file = "../../data/exp4-2/4-2-3/file_names_exp4-2-3.csv"

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "num of pitch"])

    path = "../../data/exp4-2/4-2-3"
    max_depth = 3  # we have three levels of loops in the original code
    create_files_4_2_3(array_of_note_numbers=array_of_note_numbers, num_files=1800, min_notes=3, max_notes=9, path=path, writer=writer, rhythm=rhythm, rest=rest)

print(f"File names have been written to {csv_file}")

# 4.2.4 จำนวน, ลำดับของ pitch, tempo และ rhythm แตกต่างกัน

In [ ]:
def create_files_4_2_4(
    array_of_note_numbers,
    num_files,
    min_notes,
    max_notes,
    path,
    writer,
    rhythm,
    rest,
    track=0,
    channel=0,
    tempo=120,
    time=0, 
    duration=2,
    duration_rest=2,
    volume=255,
):
    generated_files = set()
    rhythm = {4:"whole", 2:"white", 1:"black", 0.5:"half", 0.25:"Quater"}
    for _ in range(num_files):
        num_notes = random.randint(min_notes, max_notes)
        note_indices = random.sample(range(len(array_of_note_numbers)), num_notes)
        # note_indices.sort()  # Ensure notes are in ascending order
        notes_prefix = "".join([NOTES[i] for i in note_indices])
        note_indices_str = "".join(map(str, note_indices))
        tempo = random.choice([60, 80, 100, 120, 140, 160, 180])

        if note_indices_str in generated_files:
            continue

        all_rhythm = []
        generated_files.add(note_indices_str)
        MyMIDI = MIDIFile(1)
        MyMIDI.addTempo(track, 0, tempo)
        rest = {
            # 4: "whole rest",
            2: "half rest",
            # 1: "Quarter rest",
            # 0.5: "eighth rest",
            # 0.25: "Sixteenth rest",
            0: "no rest",
        }
        all_rest = []
        time = 0
        for idx in note_indices:
            duration = random.choice([4, 2, 1, 0.5, 0.25])
            # duration_rest = random.choice([4, 2, 1, 0.5, 0.25])
            duration_rest = random.choice([2, 0])
            MyMIDI.addNote(
                track, channel, array_of_note_numbers[idx] + 48, time, duration, volume
            )
            time += duration
            all_rhythm.append(rhythm[duration])

            time += duration_rest
            all_rest.append(rest[duration_rest])



        midi_file_name = f"{path}/midi_{notes_prefix+note_indices_str}.mid"
        with open(midi_file_name, "wb") as output_file:
            MyMIDI.writeFile(output_file)
        
        fs(f'{path}/midi_{notes_prefix+note_indices_str}.mid', f'{path}/wav_{notes_prefix+note_indices_str}.wav')
        label_indices = ",".join(map(str, note_indices))
        writer.writerow([f"wav_{notes_prefix+note_indices_str}.wav", f"[{label_indices}]", tempo, all_rest, num_notes, all_rest])


In [ ]:
# โน้ต 3 ตัว
rhythm = "all white"
rest = "all half rest"
# ระบุเส้นทางไปยังโฟลเดอร์ที่มีไฟล์ที่คุณต้องการเก็บข้อมูล
folder_path = "../../data/exp4-2/4-2-4"
# สร้างไฟล์ CSV สำหรับเก็บชื่อไฟล์
csv_file = "../../data/exp4-2/4-2-4/file_names_exp4-2-4.csv"

with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['File Name', 'label', 'tempo', 'rhythm', "num of pitch"])

    path = "../../data/exp4-2/4-2-4"
    max_depth = 3  # we have three levels of loops in the original code
    create_files_4_2_4(array_of_note_numbers=array_of_note_numbers, num_files=1800, min_notes=3, max_notes=9, path=path, writer=writer, rhythm=rhythm, rest=rest)

print(f"File names have been written to {csv_file}")